# 추천시스템
- 하나의 콘텐츠를 선택했을 때, 선택된 컨텐츠와 연관된 추천 컨텐츠를 제공하는 서비스 시스템
- 대표적으로 사용하는 데이터
    1. 사용자가 선택한 컨텐츠
    2. 사용자가 실제 구매와 연관된 액션을 했는가 여부
    3. 사용자가 선택한 컨텐츠에 내린 평가 데이터
    4. 사용자가 스스로 작성한 자신의 취향
    5. 사용자가 무엇을 클릭했는지...

## 1. 추천시스템 유형
- 협업 필터링 vs 콘텐츠 기반 필터링
    1. 협업 필터링: 최근접이웃 협업 vs 잠재요인 협업
        + 협업필터링이란, 사용자-아이템 평점 매트릭스와 같은 축적된 사용자 행동 데이터 기반으로 사용자가 아직 평가하지 않은 아이템을 '예측 평가'하는 것
        + 최근접 이웃 협업, 잠재요인 협업 모두 사용자-평점 매트릭스만으로 사용자에게 추천함
        + index는 user, columns는 item1, item2 등으로 구성되어야함. pandas.pivot_table() 이용해 변경해야함
        + 예시. 영화 {범죄도시1: 평점 8.5점, 범죄도시2: 평점 9점, 범죄도시3: 평점 8점} => 범죄도시 4: 평점 8.75로 예측
        + 최근접이웃 협업: 사용자 기반 vs 아이템 기반
            * 사용자 기반: 당신과 비슷한 고객들이 다음 상품도 구매했습니다.
            * 아이템 기반: 이 상품을 선택한 다른 고개들은 다음 상품도 구매했습니다.
        + 잠재요인 협업: 사용자-아이템 평점 매트리스 속에 숨어 있는 잠재 요인을 추출해 추천하는 기법
        
    2. 콘텐츠 기반 필터링: 사용자가 특정 아이템을 매우 선호하는 경우, 그 아이템과 비슷한 콘텐츠를 가진 다른 장르 아이템 추천


### 잠재요인 행렬 분해 코드
- SGD를 이용해 행렬분해를 수행
- 분해하려는 행렬 R
- P와 Q로 분해 후 다시 P와 Q.T의 내적으로 예측행렬을 만듬

In [5]:
import numpy as np

# 원본 행렬 R 생성, 분해 행렬 P와 Q 초기화, 잠재 요인 차원 K는 3으로 설정
R = np.array([[4, np.NaN, np.NaN, 2, np.NaN],
              [np.NaN, 5, np.NaN, 3, 1],
              [np.NaN, np.NaN, 3, 4, 4],
              [5, 2, 1, 2, np.NaN]])

num_users, num_items = R.shape
K=3

# P와 Q 행렬의 크기를 지정하고 정규분포를 가진 임의의 값으로 입력
np.random.seed(1)
P = np.random.normal(scale=1./K, size=(num_users, K))
Q = np.random.normal(scale=1./K, size=(num_items, K))

In [6]:
from sklearn.metrics import mean_squared_error

def get_rmse(R, P, Q, non_zeros):
    error = 0
    # 두 개의 분해된 행렬 P와 Q.T의 내적으로 예측 R 행렬 생성
    full_pred_matrix = np.dot(P, Q.T)

    # 실제 R 행렬에서 Null이 아닌 값의 위치 인덱스 추출해 실제 R 행렬과 예측 행렬의 RMSE 추출
    x_non_zero_index = [non_zero[0] for non_zero in non_zeros]
    y_non_zero_index = [non_zero[1] for non_zero in non_zeros]
    R_non_zeros = R[x_non_zero_index, y_non_zero_index]
    full_pred_matrix_non_zeros = full_pred_matrix[x_non_zero_index, y_non_zero_index]
    mse = mean_squared_error(R_non_zeros, full_pred_matrix_non_zeros)
    rmse = np.sqrt(mse)

    return rmse

In [7]:
# R > 0인 행 위치, 열 위치, 값을 non_zeros 리스트에 저장
non_zeros = [(i, j , R[i, j]) for i in range(num_users) for j in range(num_items) if R[i, j] > 0]

steps = 1000
lr = 0.01
r_lambda = 0.01

# SGD 기법으로 P와 Q 매트릭스를 계속 업데이트
for step in range(steps):
    for i, j, r in non_zeros:
        # 실제 값과 예측 값의 차이인 오류 값 구하기
        eij = r - np.dot(P[i, :], Q[j, :].T)
        # Regularlization을 반영한 SGD 업데이트 공식 적용
        P[i, :] = P[i, :] + lr*(eij*Q[j, :] - r_lambda*P[i, :])
        Q[j, :] = Q[j, :] + lr*(eij*P[i, :] - r_lambda*Q[j, :])
        rmse = get_rmse(R, P, Q, non_zeros)
        if step % 50 == 0:
            print(f"[행렬분해 중] iteration step: {step}, rmse: {rmse}")

[행렬분해 중] iteration step: 0, rmse: 3.261355059488935
[행렬분해 중] iteration step: 0, rmse: 3.26040057174686
[행렬분해 중] iteration step: 0, rmse: 3.253984404542389
[행렬분해 중] iteration step: 0, rmse: 3.2521583839863624
[행렬분해 중] iteration step: 0, rmse: 3.252335303789125
[행렬분해 중] iteration step: 0, rmse: 3.251072196430487
[행렬분해 중] iteration step: 0, rmse: 3.2492449982564864
[행렬분해 중] iteration step: 0, rmse: 3.247416477570409
[행렬분해 중] iteration step: 0, rmse: 3.241926055455223
[행렬분해 중] iteration step: 0, rmse: 3.2400454107613084
[행렬분해 중] iteration step: 0, rmse: 3.240166740749792
[행렬분해 중] iteration step: 0, rmse: 3.2388050277987723
[행렬분해 중] iteration step: 50, rmse: 0.5003190892212749
[행렬분해 중] iteration step: 50, rmse: 0.5001616291326989
[행렬분해 중] iteration step: 50, rmse: 0.4989960120257809
[행렬분해 중] iteration step: 50, rmse: 0.498848345014583
[행렬분해 중] iteration step: 50, rmse: 0.4989518925663175
[행렬분해 중] iteration step: 50, rmse: 0.4983323683009099
[행렬분해 중] iteration step: 50, rmse: 0.4984148489378

In [8]:
# 분해된 P와 Q 함수를 P*Q.T로 예측행렬 만들기
pred_matrix = np.dot(P,Q.T)
print('예측행렬:\n', np.round(pred_matrix, 3))

예측행렬:
 [[3.991 0.897 1.306 2.002 1.663]
 [6.696 4.978 0.979 2.981 1.003]
 [6.677 0.391 2.987 3.977 3.986]
 [4.968 2.005 1.006 2.017 1.14 ]]
